# <div align="center" style="color: #ff5733;">Data Requested By Annastasia for Monthly Reporting</div>

In [1]:
# %% [markdown]
# # Jupyter Notebook Loading Header
#
# This is a custom loading header for Jupyter Notebooks in Visual Studio Code.
# It includes common imports and settings to get you started quickly.

# %% [markdown]
## Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
import os
path = r'C:\Users\DwaipayanChakroborti\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = path
client = bigquery.Client(project='prj-prod-dataplatform')

# %% [markdown]
## Configure Settings
# Set options or configurations as needed
# Example: pd.set_option('display.max_columns', None)

In [2]:
# Year and Month for which data is extracted
data_year = 2024
data_month = 11

In [3]:
# How many Applications applied to each month

sq = """
# loan applied
SELECT 
    DATE_TRUNC( startApplyDateTime,MONTH) as mm, 
    case when reloan_flag = 1  and loantype not like 'FLEXUP' then 'Reloan'
         when loantype = 'FLEXUP' and new_loan_type = 'Flex-up' and reloan_flag = 0 and flagDisbursement = 1 then 'Flex-up' 
         
         else new_loan_type end as LoanProduct,
    count (distinct digitalLoanAccountId) as StartedApps
FROM 
    `prj-prod-dataplatform.risk_credit_mis.loan_master_table` 

group by 1,2
order by 1 desc,2;
"""

StartedApps = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The rows and columns in StartedApps are:\t {StartedApps.shape}")

Job ID 12016a0e-9c45-4b21-85da-48833513b2f0 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The rows and columns in StartedApps are:	 (226, 3)


In [4]:
# How many submitted applications to each month
sq = """
# loan Submitted
SELECT 
    DATE_TRUNC((case when loantype = 'FLEXUP' and new_loan_type = 'Flex-up' and reloan_flag = 0 and flagDisbursement = 1 then startApplyDateTime else termsAndConditionsSubmitDateTime end),MONTH) as mm,  
    case when reloan_flag = 1 and loantype not like 'FLEXUP'then 'Reloan'
         when loantype = 'FLEXUP' and new_loan_type = 'Flex-up' and reloan_flag = 0 and flagDisbursement = 1 then 'Flex-up' 
                  else new_loan_type end as LoanProduct,
    count (distinct digitalLoanAccountId) as SubmittedApps
FROM 
    `prj-prod-dataplatform.risk_credit_mis.loan_master_table` 
group by 1,2
order by 1 desc,2;
"""

SubmittedApps = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The rows and columns of SubmittedApps are:\t {SubmittedApps.shape}")
SubmittedApps.sample(10)

Job ID f2e805e5-d5d6-4c9c-b3b7-b500d657d05c successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The rows and columns of SubmittedApps are:	 (224, 3)


,mm,LoanProduct,SubmittedApps
12,2024-11-01,SIL Competitor,2764
104,2023-08-01,SIL-Instore,3918
181,2022-05-01,SIL-Instore,25
131,2023-03-01,Flex-up,5
139,2023-02-01,Reloan,90
100,2023-08-01,Flex,4113
129,2023-03-01,Big Loan,22
187,2022-03-01,Flex,2894
190,2022-03-01,SIL-Online(discontinued),394
68,2024-02-01,SIL-Instore,7082


In [5]:
# Approved Loans for each month

sq = """
# Approved loans
SELECT 
DATE_TRUNC((case when loantype = 'FLEXUP' and new_loan_type = 'Flex-up' and reloan_flag = 0 and flagDisbursement = 1 then startApplyDateTime 
                  when reloan_flag = 1 and loantype not like 'FLEXUP' then startApplyDateTime else decision_date end),MONTH) as mm,
        case when reloan_flag = 1 and loantype not like 'FLEXUP'then 'Reloan'
         when loantype = 'FLEXUP' and new_loan_type = 'Flex-up' and reloan_flag = 0 and flagDisbursement = 1 then 'Flex-up' 
                  else new_loan_type end as LoanProduct,
    count (distinct digitalLoanAccountId) as ApprovedApps
FROM 
    `prj-prod-dataplatform.risk_credit_mis.loan_master_table`
where 
(case when loantype = 'FLEXUP' and new_loan_type = 'Flex-up' and reloan_flag = 0 and flagDisbursement = 1 then flagDisbursement 
      when reloan_flag = 1 and loantype not like 'FLEXUP' then flagDisbursement  else flagApproval end) = 1
group by 1,2
order by 1 desc,2
;
"""

ApprovedApps = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The rows and columns in ApprovedApps are:\t {ApprovedApps.shape}")
ApprovedApps.head(10)

Job ID 019cf847-f46a-4e2d-b885-f409d548ce66 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The rows and columns in ApprovedApps are:	 (186, 3)


,mm,LoanProduct,ApprovedApps
0,2024-12-01,ACL TSA,34
1,2024-12-01,Flex-up,33
2,2024-12-01,Quick,215
3,2024-12-01,Reloan,32
4,2024-12-01,SIL Competitor,53
5,2024-12-01,SIL Repeat,1
6,2024-12-01,SIL ZERO,120
7,2024-12-01,SIL-Instore,1364
8,2024-11-01,ACL TSA,604
9,2024-11-01,Flex-up,322


In [6]:
# Booked Loans for each month

sq = """
#Booked loans
SELECT 
DATE_TRUNC( disbursementDateTime,MONTH) as mm, 
            case when reloan_flag = 1 and loantype not like 'FLEXUP' and flagDisbursement = 1 then 'Reloan'
         when loantype = 'FLEXUP' and new_loan_type = 'Flex-up' and reloan_flag = 0 and flagDisbursement = 1 then 'Flex-up' 
                  else new_loan_type end as LoanProduct,
    count (distinct digitalLoanAccountId) as BookedApps
FROM 
    `prj-prod-dataplatform.risk_credit_mis.loan_master_table` 
    where flagDisbursement = 1

group by 1,2
order by 1 desc,2
;
"""

BookedApps = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The rows and columns in BookedApps are:\t {BookedApps.shape}")
BookedApps.head(10)

Job ID e34795e7-acbd-4ce3-93d7-f83e8219b764 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The rows and columns in BookedApps are:	 (184, 3)


,mm,LoanProduct,BookedApps
0,2024-12-01,ACL TSA,37
1,2024-12-01,Flex-up,54
2,2024-12-01,Quick,211
3,2024-12-01,Reloan,32
4,2024-12-01,SIL Competitor,54
5,2024-12-01,SIL Repeat,1
6,2024-12-01,SIL ZERO,117
7,2024-12-01,SIL-Instore,1327
8,2024-11-01,ACL TSA,582
9,2024-11-01,Flex-up,435


In [7]:
# Get the total booked amount for a month

sq = """
# Booked Amount
SELECT 
DATE_TRUNC( disbursementDateTime,MONTH) as mm, 
    case when reloan_flag = 1 and loantype not like 'FLEXUP' and flagDisbursement = 1 then 'Reloan'
         when loantype = 'FLEXUP' and new_loan_type = 'Flex-up' and reloan_flag = 0 and flagDisbursement = 1 then 'Flex-up' 
                  else new_loan_type end as LoanProduct,
    sum (disbursedLoanAmount) as BookedAmt
FROM 
    `prj-prod-dataplatform.risk_credit_mis.loan_master_table` 
     where flagDisbursement = 1
group by 1,2
order by 1 desc,2
;
"""
BookedAmt = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The rows and columns in BookedAmt are:\t {BookedAmt.shape}")
BookedAmt.head(10)

Job ID 8048470e-e032-4e4c-a75d-946b288c63bb successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The rows and columns in BookedAmt are:	 (184, 3)


,mm,LoanProduct,BookedAmt
0,2024-12-01,ACL TSA,185000.00
1,2024-12-01,Flex-up,2174215.31
2,2024-12-01,Quick,2622000.00
3,2024-12-01,Reloan,619619.48
4,2024-12-01,SIL Competitor,787634.00
5,2024-12-01,SIL Repeat,19301.00
6,2024-12-01,SIL ZERO,1795426.00
7,2024-12-01,SIL-Instore,13221225.00
8,2024-11-01,ACL TSA,2910000.00
9,2024-11-01,Flex-up,15553621.72


In [8]:
BookedAmt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 184 entries, 0 to 183
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   mm           184 non-null    datetime64[us]
 1   LoanProduct  184 non-null    object        
 2   BookedAmt    184 non-null    float64       
dtypes: datetime64[us](1), float64(1), object(1)
memory usage: 4.4+ KB


In [9]:
raw_data = StartedApps.merge(SubmittedApps,how='outer').merge(ApprovedApps,how='outer').merge(BookedApps,how='outer').merge(BookedAmt,how='outer')

In [10]:
raw_data.tail(10)

,mm,LoanProduct,StartedApps,SubmittedApps,ApprovedApps,BookedApps,BookedAmt
218,2024-11-01,SIL ZERO,2311,2296,1301,1294,1.947609e+07
219,2024-11-01,SIL-Instore,19564,19339,11972,11923,1.157503e+08
220,2024-12-01,ACL TSA,51,43,34,37,1.850000e+05
221,2024-12-01,Flex-up,33,33,33,54,2.174215e+06
222,2024-12-01,Quick,3632,3027,215,211,2.622000e+06
223,2024-12-01,Reloan,40,34,32,32,6.196195e+05
224,2024-12-01,SIL Competitor,66,62,53,54,7.876340e+05
225,2024-12-01,SIL Repeat,1,1,1,1,1.930100e+04
226,2024-12-01,SIL ZERO,204,204,120,117,1.795426e+06
227,2024-12-01,SIL-Instore,2117,2094,1364,1327,1.322122e+07


In [11]:
print(f'{data_year}-{data_month:02d}-01')

2024-11-01


In [12]:
raw_data_current_month = raw_data[(raw_data['mm']>='2022-08-01') &(raw_data['mm']<=f'{data_year}-{data_month:02d}-01')]

In [13]:
raw_data_current_month

,mm,LoanProduct,StartedApps,SubmittedApps,ApprovedApps,BookedApps,BookedAmt
55,2022-08-01,Flex,5558,4732,594,599,1.198500e+07
56,2022-08-01,Quick,43764,30660,1910,1926,3.300100e+07
57,2022-08-01,SIL-Instore,967,956,456,457,1.137941e+07
58,2022-08-01,SIL-Online(discontinued),1192,652,41,41,1.197499e+06
59,2022-09-01,Big Loan,13,12,<NA>,<NA>,NaN
...,...,...,...,...,...,...,...
215,2024-11-01,Reloan,705,671,644,644,1.403602e+07
216,2024-11-01,SIL Competitor,2783,2764,1950,1948,2.418730e+07
217,2024-11-01,SIL Repeat,8,8,7,7,8.773900e+04
218,2024-11-01,SIL ZERO,2311,2296,1301,1294,1.947609e+07


### Loan Account Tagging

In [14]:
sq="""
# Loan Tagging
SELECT
    loanAccountNumber,
    Case when reloan_flag = 1 and loantype not like 'FLEXUP' and flagDisbursement = 1 then 'Reloan'
         when loantype = 'FLEXUP' and new_loan_type = 'Flex-up' and reloan_flag = 0 and flagDisbursement = 1 then 'Flex-up' 
                  else new_loan_type end as LoanProduct,
    disbursementDateTime,
    (CASE WHEN new_loan_type = 'Flex-up' then
    LAG(new_loan_type) OVER (PARTITION BY customerId ORDER BY disbursementDateTime) END) AS OriginalLoanProduct
  FROM
    `prj-prod-dataplatform.risk_credit_mis.loan_master_table`
   where flagDisbursement=1
;
"""
loan_type = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The rows and columns in loan_type are: {loan_type.shape}")
loan_type.tail(10)

Job ID 588db3ab-dfa3-448a-a804-ae8008c18963 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The rows and columns in loan_type are: (247891, 4)


,loanAccountNumber,LoanProduct,disbursementDateTime,OriginalLoanProduct
247881,60830277330011,SIL Competitor,2024-11-15 18:54:15,None
247882,60830291690013,SIL Competitor,2024-11-16 12:42:13,None
247883,60830304520012,SIL Competitor,2024-11-16 18:32:11,None
247884,60830323720011,SIL Competitor,2024-11-17 14:24:35,None
247885,60830330880014,SIL Competitor,2024-11-17 17:15:21,None
247886,60830352940016,SIL Competitor,2024-11-18 14:43:57,None
247887,60830356470011,SIL Competitor,2024-11-18 16:41:28,None
247888,60830377240015,SIL Competitor,2024-11-19 13:02:17,None
247889,60830409750019,SIL Competitor,2024-11-20 15:50:09,None
247890,60830414730016,SIL Competitor,2024-11-20 18:37:03,None


In [15]:
loan_type[loan_type['loanAccountNumber']=='60828136490016']

,loanAccountNumber,LoanProduct,disbursementDateTime,OriginalLoanProduct
123047,60828136490016,SIL-Instore,2024-09-01 12:25:12,None


In [16]:
Loan_Acs_current_month = loan_type[(loan_type['disbursementDateTime'].dt.month == data_month) & (loan_type['disbursementDateTime'].dt.year == data_year)]

In [17]:
Loan_Acs_current_month = Loan_Acs_current_month[['loanAccountNumber', 'LoanProduct','OriginalLoanProduct']]

#### VAS

In [18]:
sq = """
# Vas 
SELECT 
    extract(year from disbursementDateTime) as year , 
    extract(month from disbursementDateTime) as month,
    Case when reloan_flag = 1 and b.loantype not like 'FLEXUP' and flagDisbursement = 1 then 'Reloan'
         when b.loantype = 'FLEXUP' and new_loan_type = 'Flex-up' and reloan_flag = 0 and flagDisbursement = 1 then 'Flex-up' 
                  else new_loan_type end as new_loan_type, 
    count(distinct  b.loanAccountNumber ) AS TotalSold,
   
   count(distinct CASE WHEN vas_flag = 'true' THEN b.loanAccountNumber END) AS SoldWithVAS
     FROM `prj-prod-dataplatform.dl_loans_db_raw.tdbk_digital_loan_application` a join
`prj-prod-dataplatform.risk_credit_mis.loan_master_table` b on a.loanAccountNumber = b.loanAccountNumber
where flagDisbursement = 1
group by 1,2 ,3 
order by 1 desc,2 desc,3
;
"""

vas_df = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The rows and columns in vas_df are:\t {vas_df.shape}")
vas_df.head(10)

Job ID 75b4b518-b116-47fb-a3d3-4a259be6ea2a successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The rows and columns in vas_df are:	 (184, 5)


,year,month,new_loan_type,TotalSold,SoldWithVAS
0,2024,12,ACL TSA,37,16
1,2024,12,Flex-up,54,22
2,2024,12,Quick,211,109
3,2024,12,Reloan,32,18
4,2024,12,SIL Competitor,54,25
5,2024,12,SIL Repeat,1,1
6,2024,12,SIL ZERO,117,65
7,2024,12,SIL-Instore,1327,901
8,2024,11,ACL TSA,582,167
9,2024,11,Flex-up,435,196


In [19]:
vas_df_current_month = vas_df[(vas_df['year']==data_year)& (vas_df['month']==data_month)]

In [21]:
with pd.ExcelWriter(f'loan_accounts_tagging_{data_year}_{data_month:02d}_v1.xlsx') as writer:
    # Write each DataFrame to a specific sheet in the Excel file
    raw_data_current_month.to_excel(writer, sheet_name='raw_data', index=False)
    Loan_Acs_current_month.to_excel(writer, sheet_name='Loan Acs', index=False)
    vas_df_current_month.to_excel(writer, sheet_name='VAS', index=False)

In [22]:
print(f'loan_accounts_tagging_{data_year}_{data_month:02d}_v1.xlsx')

loan_accounts_tagging_2024_11_v1.xlsx
